In [19]:
import PIL as Image
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset

In [40]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5), (0.5, ))
])

train = datasets.CIFAR10(root="./data", train=True, transform=transform)
test = datasets.CIFAR10(root="./data", train=False, transform=transform)

In [41]:
train_data = DataLoader(dataset=train, batch_size=32, shuffle=True)
test_data = DataLoader(dataset=test, batch_size=32, shuffle=True)

In [2]:
# Building a simple classifier to classify the cifr10 data
# CIFR10 data has the 10 classes
import torch
import torch.nn as nn
import torch.optim as optim


In [48]:
class CIFR10Classifier(nn.Module):
    def __init__(self):
        super(CIFR10Classifier, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1, stride=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1, stride=2)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = None # Place holder for fully connected layer 1
        self.fc2 = nn.Linear(256, 10)
    
    def forward(self, inputs):
        x = self.pool(torch.relu(self.conv1(inputs)))
        x = self.pool(torch.relu(self.conv2(x)))

        if self.fc1 is None:
            flattened_shape = x.view(x.size(0), -1).size(1)
            self.fc1 = nn.Linear(flattened_shape, 256).to(x.device)
        
        x = x.view(x.size(0), -1)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x


In [49]:
model = CIFR10Classifier()
optimizer = optim.Adam(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [60]:
def train_model(model, train, optimizer, criterion, epochs=10):
    model.train()
    for epoch in range(epochs):
        running_loss= 0.0
        for inputs, label in train:
            inputs, label = inputs.to(device), label.to(device)
            y_pred = model(inputs)
            loss = criterion(y_pred, label)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"Epoch {epoch + 1}/{epochs}, Loss: {running_loss / len(train):.4f}")


In [61]:
train_model(model=model, train=train_data, optimizer=optimizer, criterion=criterion)

Epoch 1/10, Loss: 1.3688
Epoch 2/10, Loss: 1.3320
Epoch 3/10, Loss: 1.3037
Epoch 4/10, Loss: 1.2883
Epoch 5/10, Loss: 1.2764
Epoch 6/10, Loss: 1.2639
Epoch 7/10, Loss: 1.2589
Epoch 8/10, Loss: 1.2491
Epoch 9/10, Loss: 1.2472
Epoch 10/10, Loss: 1.2498


In [62]:
x =torch.rand(2, 2)

In [64]:
x

tensor([[0.8144, 0.7382],
        [0.4763, 0.5298]])

In [65]:
print(torch.max(x, 1))

torch.return_types.max(
values=tensor([0.8144, 0.5298]),
indices=tensor([0, 1]))


In [ ]:
# Evaluation function

def EvalauteModel(model, test, criterion):
    model.eval()
    correct = 0
    total = 0
    test_loss = 0.0
    iteration = 0
    with torch.no_grad():
        for inputs, label in test:
            y_pred = model(inputs)
            if iteration % 1000 == 0:
                print(y_pred)
            loss = criterion(y_pred, label)
            test_loss += loss.item()

            # Accuracy Calculation
            _, predicted = torch.max(y_pred, 1)
            total += label.size(0)
            correct += (predicted == label).sum().item()
            iteration += 1
        accuracy = 100 * correct / total
        print("Accuracy: ", accuracy)

In [80]:
EvalauteModel(model=model, test=test_data, criterion=criterion)

Accuracy:  53.49


In [ ]:
epochs = 10

for i in range(epochs):
    model.train()
    y_pred = model(train)
    optimizer.zero_grad()
    loss = criterion(y_pred, labe)